In [34]:
#This is basically just the code from https://zenodo.org/record/34487, set up to integrate the FLRW luminosity distances numerically

import numpy as np
from scipy import interpolate, linalg, optimize, integrate
from optparse import OptionParser
import sys
from collections import OrderedDict
import pickle
import time

In [35]:
c = 299792.458 # km/s
H0 = 70 #(km/s) / Mpc

N=740 ; # Number of SNe



In [66]:
DET=4 
REVB = False #reverse the pecuiar velocity and other corrections to mB?

In [67]:
#Get this data from https://zenodo.org/record/34487 and https://github.com/cmbant/CosmoMC/blob/master/data/jla_lcparams.txt
Z = np.load( 'Dipole_JLA/SNMLE/JLA.npy' )
jlarr = np.genfromtxt('Dipole_JLA/jla_likelihood_v6/data/jla_lcparams.txt', skip_header=1)
ZHEL = jlarr.transpose()[2]
ZCMB = jlarr.transpose()[1]
if REVB:
    Z[:,1] = Z[:,1] - jlarr[:,-1]
COVd = np.load( 'Dipole_JLA/SNMLE/covmat/stat.npy' )
for i in [ "cal", "model", "bias", "dust", "sigmaz", "sigmalens", "nonia" ]:#"pecvel" is excluded
#Notice the lack of "host" covariances - we don't include the mass-step correction.
        COVd += np.load( 'Dipole_JLA/SNMLE/covmat/'+i+'.npy' )

In [38]:
def S(Zc, OM, OL, Zh=None):
    OK = 1.-OM-OL
    def I(z):
        return 1./np.sqrt(OM * (1. + z) ** 3. + OL + OK * (1. + z) ** 2.)
    if OK == 0:
        integ = integrate.quad(I, 0, Zc)[0]
    elif OK > 0:
        integ = (1. / OK) ** (0.5) * np.sinh(integrate.quad(I, 0, Zc)[0] * OK ** (0.5))
    elif OK < 0:
        integ = (1. / -OK) ** (0.5) * np.sin(integrate.quad(I, 0, Zc)[0] * (-OK) ** (0.5))
    if Zh is not None:
        return (1.+Zh)*integ
    return (1.+Zc)*integ

In [39]:
def dLZ(Zc, OM, OL, Zh=None):
    if Zh is not None:
        return np.hstack([S(zc, OM, OL, zh) for zc, zh in zip(Zc, Zh)])
    return np.hstack([S(z, OM, OL) for z in Zc])

def MU( OM, OL ):
        return 5*np.log10( c/H0 * dL(OM,OL) ) + 25.

def MUZ(Zc, OM, OL, Zh=None):
        k = 5.*np.log10( c/H0 * dLZ(Zc, OM,OL, Zh) ) + 25.   
        if np.any(np.isnan(k)):
                print 'Fuck', OM, OL
                k[np.isnan(k)] = 63.15861331456834
        return k


In [40]:
def COV( A , B , VM, VX, VC , RV=0): # Total covariance matrix
        block3 = np.array( [[VM + VX*A**2 + VC*B**2,    -VX*A, VC*B],
                                                                                                 [-VX*A , VX, 0],
                                                                                                 [ VC*B ,  0, VC]] )
        ATCOVlA = linalg.block_diag( *[ block3 for i in range(N) ] ) ;
        
        if RV==0:
                return np.array( COVd + ATCOVlA );
        elif RV==1:
                return np.array( COVd );
        elif RV==2:
                return np.array( ATCOVlA );

def RES( OM, OL , A , B , M0, X0, C0 ): #Total residual, \hat Z - Y_0*A
        Y0A = np.array([ M0-A*X0+B*C0, X0, C0 ])
        if DET==1:
                mu = MUZ(Z[:,0], OM, OL)
        elif DET==2:
                mu = MUZ(ZCMB, OM, OL)
        elif DET==3:
                mu = MUZ(ZCMB, OM, OL, ZHEL)
        elif DET==4:
                mu = MUZ(ZHEL, OM, OL)   
        return np.hstack( [ (Z[i,1:4] -np.array([mu[i],0,0]) - Y0A ) for i in range(N) ] )  

In [41]:
def m2loglike(pars , RV = 0):
        if RV != 0 and RV != 1 and RV != 2:
                raise ValueError('Inappropriate RV value')
        else:
                cov = COV( *[ pars[i] for i in [2,5,9,4,7] ] )
                try:
                        chol_fac = linalg.cho_factor(cov, overwrite_a = True, lower = True ) 
                except np.linalg.linalg.LinAlgError: # If not positive definite
                        return +13993*10.**20 
                except ValueError: # If contains infinity
                        return 13995*10.**20
                res = RES( *[ pars[i] for i in [0,1,2,5,8,3,6] ] )

                #Dont throw away the logPI part.
                part_log = 3*N*np.log(2*np.pi) + np.sum( np.log( np.diag( chol_fac[0] ) ) ) * 2
                part_exp = np.dot( res, linalg.cho_solve( chol_fac, res) )

                if pars[0]<0 or pars[0]>1.5 or pars[1]<-.50 or pars[1]>1.5 \
                        or pars[4]<0 or pars[7]<0 or pars[9]<0:
                        part_exp += 100* np.sum(np.array([ _**2 for _ in pars ]))
                        # if outside valid region, give penalty

                if RV==0:
                        m2loglike = part_log + part_exp
                        return m2loglike 
                elif RV==1: 
                        return part_exp 
                elif RV==2:
                        return part_log 

In [42]:
bounds = ( (0,1.5),(-0.5,1.5),
                        (None,None),(None,None),(0,None),
                        (None,None),(None,None),(0,None),
                        (None,None),(0,None) )

In [50]:
#Just initial guess values taken from Nielsen at al 2015 code, not the actual results of this code
pre_found_best = np.array([  3.40658319e-01,   5.68558786e-01,   1.34469382e-01,
                                                         3.84466029e-02,   8.67848219e-01,   3.05861386e+00,
                                                         -1.59939791e-02,   5.04364259e-03,  -1.90515806e+01,
                                                  1.17007078e-02])

pre_found_noacc = np.array([  6.84438318e-02,   3.42219159e-02,   1.32357422e-01,
                                                         3.26703396e-02,   8.67993385e-01,   3.04503841e+00,
                                                         -1.33181840e-02,   5.04076126e-03,  -1.90062602e+01,
                                                  1.19991540e-02])

bnds = ( (0,1.5),(-0.5,1.5),
                        (None,None),(None,None),(0,None),
                        (None,None),(None,None),(0,None),
                        (None,None),(0,None) )

In [68]:
MLE = optimize.minimize(m2loglike, pre_found_best, method = 'SLSQP', tol=10**-10, bounds=bnds)

In [69]:
print MLE

  status: 0
 success: True
    njev: 18
    nfev: 271
     fun: -196.62233478820235
       x: array([  2.70420434e-01,   4.28716546e-01,   1.33381307e-01,
         3.98337439e-02,   8.68214059e-01,   3.02611640e+00,
        -1.46107935e-02,   5.06379117e-03,  -1.90409420e+01,
         1.24246508e-02])
 message: 'Optimization terminated successfully.'
     jac: array([  2.44140625e-04,   3.05175781e-05,   1.37329102e-03,
        -3.05175781e-05,  -1.22070312e-04,  -3.05175781e-05,
        -7.93457031e-04,   1.58691406e-03,   1.83105469e-04,
         4.57763672e-04,   0.00000000e+00])
     nit: 18


In [70]:
# Constraint fucntions for fits (constraint is func == 0
def m2NOacc( pars ):
        return pars[0]/2. - pars[1]

In [71]:
MLENoAcc = optimize.minimize(m2loglike, pre_found_best, method = 'SLSQP', constraints = ({'type':'eq', 'fun':m2NOacc}, ), tol=10**-10, bounds=bnds)

In [72]:
print MLENoAcc

  status: 0
 success: True
    njev: 22
    nfev: 311
     fun: -191.09883702929301
       x: array([  6.82592692e-02,   3.41296346e-02,   1.31877774e-01,
         3.56324170e-02,   8.68286376e-01,   3.01232872e+00,
        -1.27422151e-02,   5.05905975e-03,  -1.90079222e+01,
         1.26310455e-02])
 message: 'Optimization terminated successfully.'
     jac: array([  1.77066345e+01,  -3.54137878e+01,   0.00000000e+00,
        -3.05175781e-05,  -9.15527344e-05,   3.05175781e-05,
         3.05175781e-05,  -4.57763672e-04,   3.05175781e-05,
         4.57763672e-04,   0.00000000e+00])
     nit: 22


In [ ]:
#note that the best -2llh values are worse than those reported in NGS 2015. But this is to be expected since the error budget is different (pecvel covariances are excluded), as well as the data. Can two fits with different data and error budgets be compared for goodness of fit? https://iopscience.iop.org/article/10.1086/518808/meta seem to think you can, and this forms the basis of the idea that peculiar velocities can just be corrected away.  

u'/lustre/hpc/icecube/mrameez/WDIR/JupyterHome'